In [2]:
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd

In [29]:
np.random.seed(0)

#  Data generation

In [45]:
#networks simulation
N = 100
D0 = np.random.randint(2,size=(N,N))
D1 = np.random.randint(2,size=(N,N))
for i in range(N):
    D0[i,i] = 0
    D1[i,i] = 0

In [48]:
M0 = np.sum(D0,axis=1)
G0 = np.linalg.solve(np.diag(M0),D0)

M1 = np.sum(D1,axis=1)
G1 = np.linalg.solve(np.diag(M1),D1)

In [49]:
F0 = (np.dot(D0,D0)>0).astype(np.int)
F1 = (np.dot(D1,D1)>0).astype(np.int)

In [33]:
#GPA simulation
gpa_0 = np.random.uniform(low= 0,high=4,size=len(D))
gpa_1 = np.random.uniform(low=0,high=4,size=len(D))

# 1. MLIM model

On fait la régression bayésienne, avec les a posteriori conjugués (inverse-$\chi^2(1)$, normale conditionnelle).

si on veut juste les means et vars, il suffit de les calculer les paramètres des posterior.

In [34]:
#priors 
#on a 4 coefficients beta sur lesquels travailler 
mu0 = np.zeros(4) #mean of the conditional normal distribution 
Q0 = np.diag(np.ones(4)) #var of the conditional normal distribution
a0 = 1 #coef of the inverse gamma
b0 = 1 #scaling coefficient of the inverse gamma

In [50]:
#construction des variables explicatives et variables cibles
Y = gpa_1
Ybar = G1.dot(Y)
X = gpa_0
Xbar = G1.dot(X)

features = np.column_stack((np.ones(len(Y)),Ybar,X,Xbar))

In [36]:
#updating parameters: computing the posterior distribution parameters
Qn = features.T.dot(features) + Q0
mun = np.linalg.solve(Qn,Q0.dot(mu0)+features.T.dot(Y))
an = a0+len(Y)/2
bn = b0 + 0.5*(Y.dot(Y)+mu0.T.dot(Q0.dot(mu0))-mun.T.dot(Qn.dot(mun)))

In [41]:
mun #moyennes des lois normales a posteriori

array([ 0.94007915,  0.4576351 ,  0.05114117, -0.04501581])

On obtient donc 
$$ \beta_0 = 0.94 , \beta_{\bar{Y}} = 0.46, \beta_x = 0.05, \beta_{\bar{x}} = -0.05 $$

In [42]:
Qn #matrice de variance covariance

array([[101.        , 178.33875869, 191.37328099, 189.65432778],
       [178.33875869, 320.44412255, 342.35845649, 338.25936969],
       [191.37328099, 342.35845649, 478.81959656, 359.74901656],
       [189.65432778, 338.25936969, 359.74901656, 361.76498852]])

Ce résultat est étrange

In [40]:
bn/(an-1) #moyenne de l'inverse-gamma

1.525464109739591

In [44]:
bn**2/((an-1)**2*(an-2)) #variance de l'inverse gamma

0.047490627553134754

# 2. Exogenous network formation

normal prior on $\alpha \sim \mathcal{N}(0,1)$

$D\sim Bernoulli(...)$

=> cas conjugué?

à déterminer: posterior (à calculer)

In [54]:
#priors
alpha = np.random.randn(4)

In [56]:
#feature engineering 
#we must create |Xi-Xj| for each (i,j) couple 

dist = np.zeros((N,N))
for i in range(N):
    for j in range(i,N):
        value = np.abs(X[i]-X[j])
        dist[i][j] = value
        dist[j][i] = value

In [57]:
#updates

# 3. Endogenous network formation

aller voir appendix

# 4. Heterogeneity in peer effects

Same as 1. with different networks and extended models